In [ ]:
!pip install --user --upgrade git+https://github.com/yihong0618/twint.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/yihong0618/twint.git to /tmp/pip-req-build-m787zpxp
  Running command git clone --filter=blob:none --quiet https://github.com/yihong0618/twint.git /tmp/pip-req-build-m787zpxp
  Resolved https://github.com/yihong0618/twint.git to commit 94be35b09cfcb6042a59fab5f1581bbbc1529f8a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 27.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.1 MB/s e

In [ ]:
!pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
import twint
import threading
import nest_asyncio
import pandas as pd
import dill
import os

nest_asyncio.apply()

In [ ]:
data = pd.read_csv('left.csv')

In [ ]:
# Create your list of users here

users = data['username'][2500:]

print(len(users))

377


In [ ]:
users.reset_index(drop=True, inplace=True)

In [ ]:
resultsMap = dict()
resultsRelations = dict()
errors = []
!mkdir temp

def getFollowingForIndexes(start, end, threadId):
    resultsMap[threadId] = dict()
    resultsRelations[threadId] = []
    if start > len(users):
        return
    end = min(len(users), end)
    filename = "temp/thread" + str(threadId) + ".csv"
    for i in range(start, end):
        # Delete exising temp file if it exists
        if os.path.exists(filename):
            os.remove(filename)
        username = users[i]
        # Configure
        c = twint.Config()
        c.Username = username
        c.Store_csv = True
        c.Output = filename
        # Run twint
        try:
            twint.run.Following(c)
        except Exception as error:
            print('Thread' + str(threadId) + ' encountered an error, skipping user:', error)
            errors.append((threadId, username, str(error)))
            continue
        # Read the output of twint and store it
        try:
            df = pd.read_csv(filename)
        except Exception as error:
            continue
        resultsMap[threadId][username] = df.username.to_list()
        for user in df.username:
            resultsRelations[threadId].append((user, username))
        if i % 50 == 0:
            print('Thread' + str(threadId) + ' ' + str(i) + '/' + str(end - start))
    print('Thread' + str(threadId) + ' finished')

In [ ]:
# Configure the number of threads to use
threadCount = 13

In [ ]:
# Clean the inputs and create empty variables
threadCount = min(len(users), threadCount)
resultsMap = dict()
resultsRelations = dict()
threads = []
errors = []
# Create threads and partition out the user range each thread is responsible for.
for threadId in range(threadCount):
    interval = max(1, len(users) // threadCount)
    start = interval * threadId
    end = interval * threadId + interval
    if threadId == threadCount - 1:
        end = len(users)
    thread = threading.Thread(target=getFollowingForIndexes, args=(start, end, threadId,))
    threads.append(thread)
    thread.start()
# Wait for each thread to complete
for thread in threads:
    thread.join()


# Combine all the results of each thread into one place
result = dict()
relations = []
for threadId in resultsMap.keys():
    result.update(resultsMap[threadId])
    relations += resultsRelations[threadId]

# Write the scraped information in binary python format using dill (more efficient)
dill.dump(result, open('followingResult14.dat', 'wb')) # .dat just means this file contains binary data not for human reading
dill.dump(relations, open('followingRelationsResult14.dat', 'wb')) # .dat just means this file contains binary data not for human reading

errorFileName = 'errors.txt'
!touch {errorFileName}
file = open(errorFileName, 'a')
file.write('\n')
for e in errors:
    file.write(str(e) + '\n')
file.close()
# threads completely executed
print("Done!")

#Run the next code blocks to convert to dataframes and .csv

Streaming output truncated to the last 5000 lines.
GamerKrill
skyrimswifty
bigclasstraitor
femboyautoparts
chkspaghetti
Pub_Hist
PPathole
PeterDiamandis
alexandrosM
MarketRebels
argoproj
NYKChannel
kubernetesio
woodhaus2
BuckSexton
awscloud
GerberKawasaki
ggreenwald
TeslaJoy
StatsTeslaApp
x0_pixi_x0x
MobbyG
AtomicNumber42
I_shiht_zu_not
hymeheredoti
MarcoCavacece1
JBabin99
Jonamarie2
motdl1
thepinkneonbird
DesertDon760
Bobdino62626
jeffreym166
Aheinle503
DavidTracey18
EdMylett
TrinaSolarUS
JustinTrudeau
shantines8
Snowden
ClayTravis
lassetur_n
chrismartenson
nikkibabiee710
infiltrateproxy
gerdliinc
marknorm
ChairmanNatto
FlyingPerret
Katerina_Mtaus
cdmilind
geertconard
RidyardMike
hemantgolechha
johnpavlovitz
rajeshzad
VincentPerret16
digitec_ch
Hu_Jia_qi
WellFitHealthS
gsinnovation
GlobalShapers
OneYoungWorld
UMassChan
wesleyan_u
BKLYNMinds
USC
FamousBuffalo
ShivamD17335770
GiaISantos
malrubius
HasanIsmaik
OwnYourBitcoin
vincentdonofrio
JonBoy46241854
NIHDirector
SecondGentleman
FLOTU

CRITICAL:root:twint.get:User:'id_str'


Streaming output truncated to the last 5000 lines.
stoolpresidente
Tyson_Fury
PardesRE
NoContextHumans
EddieHearn
RealSkipBayless
lexfridman
DougPolkVids
RealCandaceO
21WIRE
hollymathnerd
ColbyCovMMA
darrentill2
summaliteraria
rickyberwick
redwhiskeypete
hugolowell
GeoRebekah
theblackcowgirl
shagamul
tomiahonen
BMeiselas
WaytowichNeil
ReverendWarnock
Kasparov63
Bebes_Scrubs
gotmtv
TimHannan
DanPriceSeattle
raykwong
chbrkr
LaKeishaChest23
meiselasb
JCLRRA
CLTgov
CharlotteTalks
DaveWeasel
JustVoteBlueUSA
harrisonjaime
RemyRose241
mstone1104
MatthewWill_iam
Blake_RESISTS
TheUSASingers
FFPNOW
ChefRonSullivan
LisaWagman3
Beav72Jim
derekjGZ
spunkkee
dcboyisangry
Hezzah
anngwg
TKinMB
natsechobbyist
Pot_Pizza_Pinot
Bill_Maxwell_
DevinCow
AVindman
IrishJAM274
danyhenriette16
SittinThinkin
HWChan555
MikeWhitaker
CAO916
johnnyventure
jimdmacdonald
marynlm
UclaBruin1998
AMarch4OurLives
LaraLengel
1FreshTake
MeidasTouchPod
itsJeffTiedrich
littledeekay
HardyMugdawan
BillBillyb02
Green_Heart
Neicy6
R

CRITICAL:root:twint.get:User:'id_str'


Streaming output truncated to the last 5000 lines.
davidncpr
johnbowman
lahnabee
jeffjackdavis
majormajor42
NFLandPolitics
hawelti
laurenthal
IJNet
indoeditor
McComsey
vandalrmc
OliverHotham
davidaconrad
tiabia
BasicBlaecGirl
MaxEnbell
beneviera
sweetwoodlands
MadeByTheMass
abcnews
VerminSupreme
DWguestbooking
MsDextra
honkbark
OHeadpull
jessetechie
BlueOtter246
ShaneTHazel
28Dave38
OverheardPorch
JradRabel
DaveR94476809
Anya_jebiga
SoniaDe16302422
LibertyApex
TN4Jo2020
RationalMomma
spudmal5
cgdeterman
Neilzfive4six
Mel4Liberty
th_nd_r
Reedy_MacReady
LPMisesVeteran
JuanyeSouth
bread8ndcircus
EricGrisham9
BrieBridschge
selenaoct2018
TheWorstLibert1
adaptivetexan
savonspage
angela__mcqueen
MarlalalalaPFFT
nimrez
TeaPartyer
LibertarianMama
RobinKingthanks
LibertySnail
sassybookworm19
kayvansoest
JCheadle4
Heathermac19891
ShiloStraw
Brittne4Liberty
Bo4Jo2020
jmkmdk2007
strugglebusohmy
JustinTMickels1
Meagan60414942
hypertarian
AndrewH55459604
KimbelSlice
diamond_may
GVoter2020
LiberAtheis

CRITICAL:root:twint.get:User:'id_str'


Thread2 encountered an error, skipping user: Cannot find twitter account with name = Toasterboast


CRITICAL:root:twint.get:User:'id_str'


Streaming output truncated to the last 5000 lines.
pamehes
fvckAk
bastieenharen
D3ckster2008
karolscript
JuliaGrzeca
AkwasiBusia
WWE124i
Chiysom
HEADEMONN
its_melai
Isabell_Krypto
BlowSmokeWithJB
08_Gang
Camilagar65
SMARTGallery2
osiwo_ose
Rektmajor
husnainaaw
wandeco2
cryptoicebag
lionardolionel
danieela_sz
leepromo
SELLOmaboea72
TieneQueSaber_Q
KieronMoorek3
Rayyann00rm
ChesleyLC
iraci_maria_
AlphaGodMich
Mystiryus
LuisTotti9
AgricolaLavinia
artofbrandys
RiverRegionArt
VioletAngel1
xiaofeiren
Melissalorigin1
_June_Solstice
miningtaxi
peterhayes123
HugoLandry
compassiom
2ndNature_3
PaulSGrimwood1
rtfavfree
_beiyo
ravneel1274
RJSOnly_
QuantumFocusEU
AbigailMaropeng
desTTilado
ArtPhotoSandL
boyle_arts
colingillen
MichaelClarida
Go_Gibberish
gerardolarreag
madamadari
milg1979
EvanNiemann
Teen_mom_Crypto
Iamlizb17
Team_1DDrive
tinderlass
DYOR_100X
lilixli
fuckerino
jiohannyssz
FernandoDecoud
RandiGraceNilsb
MABarrabia
JShinnCreative
j2slikk
amrowner
josie_maskell
nutting_betterr
CirrusSto

In [ ]:
result = dict()
relations = []
for threadId in resultsMap.keys():
    result.update(resultsMap[threadId])
    relations += resultsRelations[threadId]

# Write the scraped information in binary python format using dill (more efficient)
dill.dump(result, open('followingResult13.dat', 'wb')) # .dat just means this file contains binary data not for human reading
dill.dump(relations, open('followingRelationsResult13.dat', 'wb')) # .dat just means this file contains binary data not for human reading

errorFileName = 'errors.txt'
!touch {errorFileName}
file = open(errorFileName, 'a')
file.write('\n')
for e in errors:
    file.write(str(e) + '\n')
file.close()
# threads completely executed
print("Done!")

larssonap
TaylorG64143008
iamgabesanchez
johnalogue
FrankConniff
theliamnissan
DnommPaul1
cindy_everitt
Michell70232420
BBWriterCRD
tcronin888
LaurenBrns
DrGetOffYourAss
Infinty1001
BarbaraLantz12
speakrighteous
janicefrye0846
defenestrate161
Mary609Oklahoma
CrlitoSantiago
DepratoNancy
ONealDennis
DeniceAllHeart
PLWSF
TheNewPragmatic
DropTheFacts
PJC_Raiders
teambluestate
doublehappi
KatzTibor
DropsOJupiter7
KnewsNut
ginamaureen01
Tweety66771
mplscleek
BalancePose
climate_alert
missimpolitic
TomMorrowisnow
smitbag9
justachemist002
DamourKaren
Sea2Sky2021
eastsider23
pattipetersonny
IamNicolette_2
KingDareDK
RobK8000
kjlevy100
demarte_s
attyforlife
D75Kelley
pastorcager
seihoosier
jac1161959
no2l
SeashellsForMe
Geovohm
MaxBoot
zonedfortruth
RockeyRozell
Delilah94014363
scratchford
TheLastSquatch
LovePar45351124
rsmithsantacruz
kevwrightwriter
GinaLew62634137
spectru71453165
Nanciliz
JJo610215
pacersq
MikeBro06688175
kuisnonduality
1DESTROYTHEGOP
KeithRo68564028
BuerckDenise
HonkyTonkBob

In [ ]:
followingMap = dill.load(open('followingResult12.dat', 'rb'))
followingRelations = dill.load(open('followingRelationsResult12.dat', 'rb'))

In [ ]:
df = pd.DataFrame(followingRelations)
df.to_csv('followingRelations12.csv',index=False)
df.head(20)

,0,1
0,khamenei_ir,Im_AliMustafa
1,justfred_ar,Im_AliMustafa
2,martinkallstrom,Im_AliMustafa
3,warclandestine,Im_AliMustafa
4,hadithoftheday,Im_AliMustafa
5,tedielts,Im_AliMustafa
6,eng_advanced,Im_AliMustafa
7,grantcardone,Im_AliMustafa
8,kroordarshan,Im_AliMustafa
9,bandreviews,Im_AliMustafa


In [ ]:
tempArr = []
for k in followingMap:
    # print(k, followingMap[k][0])
    tempArr.append([k, str(followingMap[k])])
df2 = pd.DataFrame(tempArr)
df2.to_csv('following_user12.csv',index=False)
df2.head(50)

,0,1
0,Im_AliMustafa,"['khamenei_ir', 'justfred_ar', 'martinkallstro..."
1,Moon1ightRose,"['dalewood', 'yourhelpfulplum', '1861x2', 'csm..."
2,Yvonne11972,"['aocpresstwo', 'truecrimegisela', 'shellenber..."
3,Michael_Psalm51,"['pets_areamazing', 'bibleinayearbot', 'oaktri..."
4,SemperAdventum,"['charles07788205', 'heather_eire', 'death6102..."
5,WickAbortions,"['virgilwalkeroma', 'william_e_wolfe', 'bradle..."
6,JustMagsMags,"['brotherjones_', 'orbplanet', 'sunnygrandma4'..."
7,JonesMurphy,"['beadbop', 'franki_watcho', 'koolkaryn', 'eji..."
8,unspinthespina1,"['macinnisval', 'jqvxyz', 'bohemianatmosp1', '..."
9,DJInsom,"['wigfritter', 'yinnyart', 'nem_graphics', 'vo..."


In [ ]:
len(df2)

963